# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:44<00:00, 20.94s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

'Title: Creality 3D Printer Filament at eBay: Buy 6, Get 4 free + free shipping\nDetails: Buy 6 spools of Creality PET-G, PLA, or Silk PLA filament and get four more for free. (Add 10 spools of your choice to your cart to get this deal, four spools will be discounted automatically during checkout.) Shop Now at eBay\nFeatures: \nURL: https://www.dealnews.com/Creality-3-D-Printer-Filament-at-eBay-Buy-6-Get-4-free-free-shipping/21725848.html?iref=rss-c196'

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [9]:
print(user_prompt)

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Jackery Solar Generator 2000 Plus w/ SolarSaga 500 X Solar Panel for $1,699 + free shipping
Details: Apply coupon code "HI1600" for a savings of $1,600, and $200 under our mention from three weeks ago. Buy Now at Jackery, Inc
Features: LFP battery with up to 4,000 charge cycles and 10 year lifespan  Charges via solar in 4 hours  Smart app control
URL: https://www.de

In [10]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [11]:
result = get_recommendations()

In [12]:
len(result.deals)

5

In [13]:
result.deals[1]

Deal(product_description="The Jackery Explorer 2000 Plus is a robust portable power station featuring a capacity of 4,086Wh alongside two SolarSaga 200W bifacial solar panels. Perfect for camping trips, outdoor events, or emergency power needs, this power station can support high-wattage devices without fuss. Its impressive 3,000W output ensures that you can run multiple devices simultaneously, providing peace of mind whether you're away from home or facing an unexpected power outage. Compact and efficient, this model is designed for those seeking reliable portable energy.", price=2499.0, url='https://www.dealnews.com/products/Jackery/Jackery-Explorer-2000-Plus-3-000-W-Portable-Power-Station-2-Solar-Saga-200-W-Solar-Panels/486971.html?iref=rss-c196')

In [14]:
from agents.scanner_agent import ScannerAgent

In [15]:
agent = ScannerAgent()
result = agent.scan()

In [19]:
from pprint import pprint
pprint(result.deals)

[Deal(product_description='The Jackery Solar Generator 2000 Plus comes with a SolarSaga 500 X solar panel and features a lithium iron phosphate (LFP) battery, capable of up to 4,000 charge cycles and a 10-year lifespan. It can be fully charged via solar energy in just 4 hours, making it an eco-friendly power solution for outdoor and emergency use. The generator is also equipped with smart app control, allowing users to monitor and manage the power station conveniently from their mobile devices.', price=1699.0, url='https://www.dealnews.com/Jackery-Solar-Generator-2000-Plus-w-Solar-Saga-500-X-Solar-Panel-for-1-699-free-shipping/21725834.html?iref=rss-c196'),
 Deal(product_description="The Jackery Explorer 2000 Plus is a formidable portable power station featuring a 4,086Wh capacity and comes bundled with two SolarSaga 200W solar panels. This power station is capable of delivering a continuous output of 3,000 watts, making it suitable for powering various appliances during outdoor advent